In [ ]:
import qiskit
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.synthesis import TwoQubitWeylDecomposition 
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import random_unitary

In [51]:
# creating a random unitary
random_unitary = random_unitary(4)
a_original = TwoQubitWeylDecomposition(random_unitary).a * 2 # Multiply by 2!
b_original = TwoQubitWeylDecomposition(random_unitary).b * 2
c_original = TwoQubitWeylDecomposition(random_unitary).c * 2


In [52]:
# B gate: e^(i*pi* (2XX+YY)/4)) -> -pi/2 rotation about xx and -pi/4 rotation about yy
b_gate = QuantumCircuit(2)
b_gate.rxx(-np.pi/2, 0,1) 
b_gate.ryy(-np.pi/4,0,1) 
B = Operator(b_gate)

In [53]:
# Defining a B-sandwich operator
middle_rotations = [0]

beta1 = np.arccos(1-4*((np.sin(b_original/2))**2)*((np.cos(c_original/2))**2)) # values from the linked paper: https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.93.020502
beta2 = np.arcsin(np.sqrt((np.cos(b_original)*np.cos(c_original))/(1-2*(np.sin(b_original/2))**2*(np.cos(c_original/2))**2)))

def B_sandwich(b_gate, beta1, beta2): # includes phase correction
    qr = QuantumRegister(2)
    circ = QuantumCircuit(qr)
    circ.append(b_gate, qr)
    circ.ry((-1+2*(c_original < 0))*a_original,qr[0])
    circ.rz(-beta2,qr[1])
    circ.ry(-beta1,qr[1])
    circ.rz(-beta2,qr[1])
    # ^ a will have positive sign if c>0 and negative if c<0, meaning that phase is accounted for
    circ.append(b_gate, qr)
    return circ

B_Sandwich_Gate = B_sandwich(b_gate, beta1, beta2)

#create B sandwich operator
B_Sandwich = Operator(B_Sandwich_Gate)

In [54]:
# New coordinates
a_experimental = TwoQubitWeylDecomposition(B_Sandwich).a * 2
b_experimental = TwoQubitWeylDecomposition(B_Sandwich).b * 2
c_experimental = TwoQubitWeylDecomposition(B_Sandwich).c * 2

In [55]:
print("Weyl Coordinates:")
print(f"Original Coordinates: (a,b,c) = {a_original:.3f}, {b_original:.3f}, {c_original:.3f}")
print(f"Decomposed Coordinates: (a,b,c) = {a_experimental:.3f}, {b_experimental:.3f}, {c_experimental:.3f}")

Weyl Coordinates:
Original Coordinates: (a,b,c) = 1.049, 0.981, -0.605
Decomposed Coordinates: (a,b,c) = 1.049, 0.981, -0.605
